<a href="https://colab.research.google.com/github/kenichikawaguchi/SD1111_colab/blob/main/SD1111_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title ⬇［SD/WebUIの起動／停止］この下の再生ボタンをクリックするとSD/WebUIが起動／停止します

# @markdown > Running on local URL:  [http&#58;//127.0.0.1:7860](http://)<br>
# @markdown > Running on public URL: [https&#58;//xxxxxxxx-xxxx-xxxx.gradio.live](http://) ←これが出たらクリック

# @markdown 再生ボタンをクリックして7分～10分程度でこの表示が出ます。2行あるリンクのうち、下の行のリンクをクリックするとWebUIが開きます。

import os

# RAMとVRAMの容量取得(GiB)
res = !free -m | grep 'Mem:' | awk '{print $2}'
if not res:
   raise ValueError("RAM容量を取得できません")
ram = round(float(res[0]) / 1024, 3)
res = !nvidia-smi --query-gpu=memory.total --format=csv,noheader,nounits
if not res:
   raise ValueError("VRAM容量を取得できません。GPUに接続していない可能性があります。「編集」メニューの「ノートブックの設定」で「ハードウェアアクセラレータ」を「GPU」に変更してください。")
vram = round(float(res[0]) / 1024, 3)
print(f"RAM : {ram}")
print(f"VRAM: {vram}")
del res

# Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

!apt -y update -qq
!wget https://mirrors.wikimedia.org/ubuntu/pool/main/g/google-perftools/libgoogle-perftools-dev_2.7-1ubuntu2_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/google-perftools_2.7-1ubuntu2_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/libgoogle-perftools4_2.7-1ubuntu2_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.7-1ubuntu2/+build/18734182/+files/libtcmalloc-minimal4_2.7-1ubuntu2_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb

!pip install torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!apt -y install -qq aria2
!pip install -q xformers==0.0.18
!pip install -q triton==2.0.0

# SD/WebUIのセットアップ
%cd /content/
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git
%mkdir /content/stable-diffusion-webui/models/Lora
%cd stable-diffusion-webui/models/Stable-diffusion

# 公式学習モデルのダウンロード
!wget -c "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt
!wget -c "https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors" -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.safetensors
!wget -c "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.vae.pt
!wget -c "https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml" -O /content/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

# でりだモデルのダウンロード
!wget -c "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/derrida_final.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/derrida_final.ckpt
!wget -c "https://huggingface.co/naclbit/trinart_derrida_characters_v2_stable_diffusion/resolve/main/autoencoder_fix_kl-f8-trinart_characters.ckpt" -O /content/stable-diffusion-webui/models/Stable-diffusion/derrida_final.vae.pt

# chilled_re-generic_v2 (2.13GB)のダウンロード
!wget -c "https://u.pontalk.com/chilled_re-generic_v2.safetensors" -O /content/stable-diffusion-webui/models/chilled_re-generic_v2.safetensors
# hyperanatomy_MegamergeV1のダウンロード
# !wget -c "https://u.pontalk.com/hyperanatomy_MegamergeV1.safetensors" -O /content/stable-diffusion-webui/models/hyperanatomy_MegamergeV1.safetensors
# dreamlike-photoreal-2.0.safetensors (2.13GB)のダウンロード
!wget -c "https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0/resolve/main/dreamlike-photoreal-2.0.safetensors" -O /content/stable-diffusion-webui/models/dreamlike-photoreal-2.0.safetensors
# BRA(Beautiful Realistic Asians) V3のダウンロード
# !wget -c "https://civitai.com/api/download/models/39685" -O /content/stable-diffusion-webui/models/BeautifulRealisticAsiansV3.safetensors
# japaneseDollLikeness_v15のダウンロード
!wget -c "https://civitai.com/api/download/models/34562" -O /content/stable-diffusion-webui/models/Lora/japaneseDollLikeness_v15.safetensors
# koreanDollLikeness_v15のダウンロード
!wget -c "https://huggingface.co/jomcs/NeverEnding_Dream-Feb19-2023/resolve/main/lora/koreanDollLikeness_v15.safetensors" -O /content/stable-diffusion-webui/models/Lora/koreanDollLikeness_v15.safetensors
# cuteGirlMix4_v10のダウンロード
!wget -c "https://huggingface.co/luxluna/cuteGirlMix4_v10/resolve/main/cuteGirlMix4_v10.safetensors" -O /content/stable-diffusion-webui/models/Lora/cuteGirlMix4_v10.safetensors
# chilled_re-generic_v3のダウンロード
# !wget -c "https://civitai.com/api/download/models/22033" -O /content/stable-diffusion-webui/models/Stable-diffusion/chilledReGenericV3_v10.safetensors
# Basil_mix_fixedのダウンロード
# !wget  -c "https://huggingface.co/nuigurumi/basil_mix/resolve/main/Basil_mix_fixed.safetensors" -O /content/stable-diffusion-webui/models/Stable-diffusion/Basil_mix_fixed.safetensors
# !wget  -c "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors" -O /content/stable-diffusion-webui/models/VAE/vae-ft-mse-840000-ema-pruned.safetensors
# MUSE_v1のダウンロード
# !wget  -c "https://civitai.com/api/download/models/15980" -O /content/stable-diffusion-webui/models/Stable-diffusion/museV1_v1.safetensors
# LOFI V1 fp16のダウンロード
# !wget  -c "https://civitai.com/api/download/models/11598" -O /content/stable-diffusion-webui/models/Stable-diffusion/lofi_V1Fp16.safetensors
# colorful_v13のダウンロード
# !wget  -c "https://civitai.com/api/download/models/10282" -O /content/stable-diffusion-webui/models/Stable-diffusion/colorful_v13.safetensors
# control_openpose-fp16のダウンロード
# !wget  -c "https://huggingface.co/webui/ControlNet-modules-safetensors/resolve/main/control_openpose-fp16.safetensors" -O /content/stable-diffusion-webui/extensions/sd-webui-controlnet/models/control_openpose-fp16.safetensors
# Googleドライブ上の設定ファイルをバックアップしランタイム上へシンボリックリンク（styles.csvは初回起動時に外部からコピーする）
sdpath     = "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion/"
configpath = sdpath + "config.json"
uicfgpath  = sdpath + "ui-config.json"
stylespath = sdpath + "styles.csv"

if os.path.isfile(configpath):
   !cp -v "{configpath}" "{sdpath}config_bak.json"
   !ln -s -v "{configpath}" "/content/stable-diffusion-webui/config.json"
if os.path.isfile(uicfgpath):
   !cp -v "{uicfgpath}" "{sdpath}ui-config_bak.json"
   !ln -s -v "{uicfgpath}" "/content/stable-diffusion-webui/ui-config.json"
if os.path.isfile(stylespath):
   !cp -v "{stylespath}" "{sdpath}styles_bak.csv"
else:
   !wget -c "https://raw.githubusercontent.com/imamurayusuke/SD1111_colab/main/styles.csv" -O "{sdpath}styles.csv"
!ln -s -v "{stylespath}" "/content/stable-diffusion-webui/styles.csv"

# Googleドライブ上に画像の出力先フォルダを作成（初回のみ）、ランタイム上からそこへシンボリックリンク
outputspath = sdpath + "outputs"
logpath     = sdpath + "log"

if not os.path.exists(outputspath):
   !mkdir -p -v "{outputspath}"
!ln -s -v "{outputspath}" "/content/stable-diffusion-webui/outputs"

if not os.path.exists(logpath):
   !mkdir -p -v "{logpath}"
!ln -s -v "{logpath}" "/content/stable-diffusion-webui/log"

# ランタイム上のExtensionsフォルダにあるput extensions here.txtをGoogleドライブにコピー（初回のみ）
extpath     = sdpath + "extensions"

if not os.path.exists(extpath):
   !mv -v "/content/stable-diffusion-webui/extensions" "{sdpath}"
else:
   !mv -v "/content/stable-diffusion-webui/extensions" "/content/stable-diffusion-webui/extensions_org"

#Extensionsフォルダへランタイム上からシンボリックリンク
!ln -s -v "{extpath}" "/content/stable-diffusion-webui/"

# ランタイム上のembeddingsフォルダにあるput extensions here.txtをGoogleドライブにコピー（初回のみ）
embedpath   = sdpath + "embeddings"

if not os.path.exists(embedpath):
   !mv -v "/content/stable-diffusion-webui/embeddings" "{sdpath}"
else:
   !mv -v "/content/stable-diffusion-webui/embeddings" "/content/stable-diffusion-webui/embeddings_org"

#embeddingsフォルダへランタイム上からシンボリックリンク
!ln -s -v "{embedpath}" "/content/stable-diffusion-webui/"

# 必ず利用するエクステンションをダウンロード（初回のみ）
if not os.path.exists(extpath + "/stable-diffusion-webui-localization-ja_JP"):
   !git clone https://github.com/Katsuyuki-Karasawa/stable-diffusion-webui-localization-ja_JP /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-ja_JP
else:
   %cd /content/stable-diffusion-webui/extensions/stable-diffusion-webui-localization-ja_JP
   !git pull 
   %cd /content/stable-diffusion-webui/models/Stable-diffusion

#日本語化拡張機能に変更があったとき、続くメッセージがすべて赤字になるのを抑止
print('\033[0m')

!sed -i -e '''/    prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/stable-diffusion-webui/launch.py
!sed -i -e 's/fastapi==0.90.1/fastapi==0.89.1/g' /content/stable-diffusion-webui/requirements_versions.txt

# SD/WebUIの起動
%cd /content/stable-diffusion-webui/
!python launch.py --share --xformers --enable-insecure-extension-access --gradio-queue --ckpt /content/stable-diffusion-webui/models/Stable-diffusion/chilled_re-generic_v2.safetensors


In [ ]:
#@title ⬇［設定のバックアップ］SD/WebUIを停止した後に実行してください

# @markdown 以下のファイルがGoogleドライブにバックアップされます。

# @markdown * 初回起動時に生成される設定ファイル（ui-config.json）

# @markdown * Settingsタブの「Apply settings」を初めてクリックしたときに生成される設定ファイル（config.json）

# @markdown 一度バックアップされれば、以降はSD/WebUIが起動時にGoogleドライブ上の設定ファイルを読み出し、設定を変更すればGoogleドライブ上の設定ファイルも更新されます。

# @markdown バックアップ先は「Colab Notebooks」－「Stable Diffusion」フォルダです。

# @markdown 実行後のメッセージの末尾が2行とも「～ are the same file」と出るようになれば、すべてGoogleドライブに保存されています。そうなれば、次回以降このコードセルを実行する必要はなくなります。
!cp -v "/content/stable-diffusion-webui/config.json" "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion"
!cp -v "/content/stable-diffusion-webui/ui-config.json" "/content/drive/MyDrive/Colab Notebooks/Stable Diffusion"

In [ ]:
#@title ⬇［再起動］SD/WebUIの再起動用コードセル

#@markdown いったん起動してから停止したSD/WebUIを再起動するときに使います。ダウンロードやインストールを省略できるため約1分で起動します。

%cd /content/stable-diffusion-webui/
!python launch.py --share --xformers --enable-insecure-extension-access --gradio-queue --ckpt /content/stable-diffusion-webui/models/Stable-diffusion/chilled_re-generic_v2.safetensors

## このノートブックについて
- このノートブックの詳細は[imamurayusuke/SD1111\_colab \- GitHub](https://github.com/imamurayusuke/SD1111_colab)をご覧ください
- 作成にあたっては以下を参考にさせていただきました
 - [camenduru/stable\-diffusion\-webui\-colab: stable diffusion webui colab](https://github.com/camenduru/stable-diffusion-webui-colab)
 - [aka\_1111\.ipynb \- Colaboratory](https://colab.research.google.com/drive/1GhvterZebPlZU4P7RcUK-U-4Y23rZpQO)

### 大ざっぱな更新履歴

- 2023/3/26：起動しなくなり修正。起動後のエラーは相変わらずです
- 2023/3/19：起動しなくなり修正。SD/WebUIが起動してリンクをクリックした際にエラーが出るのが解消できていませんが、使用に問題はないものと思われるところまで直せました。引き続き時間を見つけてエラーへの対処を行っていきます